In [2]:
import google.auth
import pandas as pd
import pytidycensus as tc

pd.set_option("display.max_columns", None)

In [3]:
import google.auth
import pandas_gbq

credentials, project = google.auth.default()
from functools import cache

from calitp_data_analysis.gcs_pandas import GCSPandas

In [4]:
@cache
def gcs_pandas():
    return GCSPandas()

In [5]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [6]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/equity_index"

In [7]:
analysis_year = 2023

In [24]:
ca_fips = "06"

In [27]:
survey_variable = "acs5"

In [9]:
with open("config.txt", "r") as file:
    api_key = file.read().strip()

In [10]:
tc.set_census_api_key(api_key)

Census API key has been set for this session.


In [28]:
# Get CA county FIPS list via the API (or use your own list)
ca_counties = tc.get_acs(
    geography="county",
    variables=["B01001_001E"],
    year=analysis_year,
    survey=survey_variable,
    state=ca_fips,
    output="wide",
)

county_codes = ca_counties["county"].unique().tolist()

Getting data from the 2019-2023 5-year ACS


In [30]:
ca_counties.sample()

,GEOID,B01001_001E,state,county,NAME,B01001_001_moe
54,06109,54873,06,109,"Tuolumne County, California",<NA>


In [31]:
# county_codes

In [69]:
# State median household income
state_income_df = tc.get_acs(
    geography="state",
    variables=["B19013_001E"],  # Median household income
    year=analysis_year,
    state=ca_fips,  # California FIPS
    output="wide",
)

Getting data from the 2019-2023 5-year ACS


In [70]:
state_income_df

,GEOID,B19013_001E,state,NAME,B19013_001_moe
0,06,96334,06,California,298.00


In [75]:
state_cutoff = int(.8 * state_income_df["B19013_001E"].iloc[0])

In [63]:
hh_income_data = []
# Median HH income at block group level
for county in county_codes:
    df = tc.get_acs(
        geography="block group",
        variables=["B19013_001"],
        year=analysis_year,
        survey=survey_variable,
        state=ca_fips,
        county=county,
        tract="*",
        output="wide",
    )
    hh_income_data.append(df)

Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting dat

In [66]:
hh_income_data_bg_df = pd.concat(hh_income_data, ignore_index=True)[
    ["GEOID", "B19013_001E"]
].rename(columns={"B19013_001E": "median_hh_income_bg"})

In [67]:
hh_income_data_bg_df.head(2)

,GEOID,median_hh_income_bg,state,county,tract,block group,NAME,B19013_001_moe
0,060014001001,250001,06,001,400100,1,"Alameda County, California",<NA>
1,060014001002,242813,06,001,400100,2,"Alameda County, California",34973.00


In [77]:
hh_size_data = []
# Median HH income at block group level
for county in county_codes:
    df = tc.get_acs(
        geography="block group",
        variables=["B25010_001"],
        year=analysis_year,
        survey=survey_variable,
        state=ca_fips,
        county=county,
        tract="*",
        output="wide",
        geometry = False,
    )
    hh_size_data.append(df)

Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting data from the 2019-2023 5-year ACS
Getting dat

In [80]:
hh_size_data_df = pd.concat(hh_size_data, ignore_index=True)[["GEOID","B25010_001E"]].rename(columns
                                                                                            ={"B25010_001E":"rounded_hh_size_bg"})

In [83]:
hh_size_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25607 entries, 0 to 25606
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   GEOID               25607 non-null  object
 1   rounded_hh_size_bg  25453 non-null  object
dtypes: object(2)
memory usage: 400.2+ KB


In [89]:
hh_size_data_df['rounded_hh_size_bg'] = hh_size_data_df['rounded_hh_size_bg'].apply(lambda x: int(float(x)) if isinstance(x, str) and x.isdigit() else x)
hh_size_data_df['rounded_hh_size_bg'] = pd.to_numeric(hh_size_data_df['rounded_hh_size_bg'], errors='coerce').astype('Int64')

TypeError: cannot safely cast non-equivalent object to int64